# E-commerce Business Analytics Dashboard

A comprehensive analysis of e-commerce sales data focusing on business performance metrics, customer satisfaction, and operational efficiency.

## Table of Contents

1. [Introduction & Business Objectives](#introduction)
2. [Data Loading & Configuration](#data-loading)
3. [Data Dictionary](#data-dictionary)
4. [Data Preparation & Transformation](#data-preparation)
5. [Business Metrics Analysis](#business-metrics)
   - [Revenue Performance Analysis](#revenue-analysis)
   - [Product Category Performance](#product-analysis)
   - [Geographic Performance Analysis](#geographic-analysis)
   - [Customer Experience Analysis](#customer-analysis)
6. [Summary of Key Observations](#summary)

---

## 1. Introduction & Business Objectives {#introduction}

This analysis provides insights into e-commerce business performance through comprehensive examination of sales data. The primary objectives are:

- **Revenue Performance**: Analyze total revenue, growth trends, and order patterns
- **Product Strategy**: Identify top-performing categories and optimization opportunities
- **Geographic Insights**: Understand regional performance variations
- **Customer Satisfaction**: Evaluate delivery performance and review metrics
- **Operational Efficiency**: Assess delivery times and fulfillment quality

### Analysis Configuration

The analysis can be configured for different time periods by adjusting the parameters below:

In [ ]:
# Analysis Configuration
ANALYSIS_YEAR = 2023
COMPARISON_YEAR = 2022
ANALYSIS_MONTH = None  # Set to specific month (1-12) or None for full year
DATA_PATH = 'ecommerce_data/'

print(f"Analysis Period: {ANALYSIS_YEAR}")
print(f"Comparison Period: {COMPARISON_YEAR}")
if ANALYSIS_MONTH:
    print(f"Month Filter: {ANALYSIS_MONTH}")
else:
    print("Month Filter: Full Year")

## 2. Data Loading & Configuration {#data-loading}

Loading all required datasets and initializing the analysis framework.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Import custom modules
from data_loader import EcommerceDataLoader, load_and_process_data
from business_metrics import BusinessMetricsCalculator, MetricsVisualizer, print_metrics_summary

# Configure display options
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("Libraries imported successfully")

In [ ]:
# Load and process all data
loader, processed_data = load_and_process_data(DATA_PATH)

# Display data loading summary
print("Data Loading Results:")
print("=" * 50)
for name, info in loader.load_summary.items():
    print(f"  {name}: {info['records']:,} records ({info['status']})")

# Display data summary
data_summary = loader.get_data_summary()
print("\nDataset Summary:")
print("=" * 50)
for dataset, info in data_summary.items():
    print(f"{dataset.upper()}:")
    print(f"  Rows: {info['rows']:,}")
    print(f"  Columns: {info['columns']}")
    print(f"  Memory: {info['memory_usage_mb']:.1f} MB")
    if info['date_range']:
        print(f"  Date Range: {info['date_range']['start'].date()} to {info['date_range']['end'].date()}")
    print()

## 3. Data Dictionary {#data-dictionary}

### Key Business Terms and Column Definitions

| Column | Description | Business Impact |
|--------|-------------|----------------|
| **order_id** | Unique identifier for each customer order | Primary key for order-level analysis |
| **price** | Item price excluding shipping | Core revenue metric |
| **freight_value** | Shipping cost for the item | Additional revenue and cost analysis |
| **order_status** | Current status of the order | Operational efficiency indicator |
| **order_purchase_timestamp** | When the order was placed | Time-based analysis and trends |
| **order_delivered_customer_date** | When order was delivered to customer | Delivery performance metric |
| **product_category_name** | Product category classification | Product strategy and inventory planning |
| **customer_state** | Customer's state location | Geographic market analysis |
| **review_score** | Customer satisfaction rating (1-5) | Customer experience indicator |

### Calculated Metrics

- **Total Revenue**: Sum of all item prices for delivered orders
- **Average Order Value (AOV)**: Average total value per order
- **Delivery Days**: Time between order placement and delivery
- **Revenue Growth**: Year-over-year percentage change in revenue
- **Customer Satisfaction**: Distribution and average of review scores

## 4. Data Preparation & Transformation {#data-preparation}

Creating the comprehensive sales dataset for analysis with configurable time filters.

In [ ]:
# Create sales dataset for analysis period
sales_data = loader.create_sales_dataset(
    year_filter=ANALYSIS_YEAR,
    month_filter=ANALYSIS_MONTH,
    status_filter='delivered'
)

print(f"Analysis Dataset Summary:")
print(f"Total Records: {len(sales_data):,}")
print(f"Unique Orders: {sales_data['order_id'].nunique():,}")
print(f"Date Range: {sales_data['order_purchase_timestamp'].min().date()} to {sales_data['order_purchase_timestamp'].max().date()}")
print(f"Total Revenue: ${sales_data['price'].sum():,.2f}")

# Display sample of the dataset - only show available columns
available_columns = ['order_id', 'price', 'purchase_year', 'purchase_month']
optional_columns = ['product_category_name', 'customer_state', 'review_score', 'delivery_days']

# Add optional columns if they exist
for col in optional_columns:
    if col in sales_data.columns:
        available_columns.append(col)

print(f"\nAvailable columns: {available_columns}")
print("\nSample Data:")
display(sales_data[available_columns].head())

In [ ]:
# Create comparison dataset if comparison year is specified
comparison_data = None
if COMPARISON_YEAR:
    comparison_data = loader.create_sales_dataset(
        year_filter=COMPARISON_YEAR,
        month_filter=ANALYSIS_MONTH,
        status_filter='delivered'
    )
    
    print(f"Comparison Dataset ({COMPARISON_YEAR}):")
    print(f"Total Records: {len(comparison_data):,}")
    print(f"Unique Orders: {comparison_data['order_id'].nunique():,}")
    print(f"Total Revenue: ${comparison_data['price'].sum():,.2f}")

# Create combined dataset for year-over-year analysis
if COMPARISON_YEAR:
    combined_data = loader.create_sales_dataset(
        month_filter=ANALYSIS_MONTH,
        status_filter='delivered'
    )
    # Filter to only include analysis and comparison years
    combined_data = combined_data[
        combined_data['purchase_year'].isin([ANALYSIS_YEAR, COMPARISON_YEAR])
    ]
else:
    combined_data = sales_data

## 5. Business Metrics Analysis {#business-metrics}

Comprehensive analysis of key business performance indicators.

In [ ]:
# Initialize metrics calculator
metrics_calc = BusinessMetricsCalculator(combined_data)

# Generate comprehensive report
business_report = metrics_calc.generate_comprehensive_report(
    current_year=ANALYSIS_YEAR,
    previous_year=COMPARISON_YEAR
)

# Print executive summary
print_metrics_summary(business_report)

### 5.1 Revenue Performance Analysis {#revenue-analysis}

Analyzing overall revenue trends, growth patterns, and key performance indicators.

In [ ]:
# Revenue metrics deep dive
revenue_metrics = business_report['revenue_metrics']

print(f"DETAILED REVENUE ANALYSIS - {ANALYSIS_YEAR}")
print("=" * 50)
print(f"Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Total Items Sold: {revenue_metrics['total_items_sold']:,}")
print(f"Average Order Value: ${revenue_metrics['average_order_value']:,.2f}")

if COMPARISON_YEAR and 'revenue_growth_rate' in revenue_metrics:
    print(f"\nYEAR-OVER-YEAR COMPARISON:")
    print(f"Revenue Growth: {revenue_metrics['revenue_growth_rate']:+.2f}%")
    print(f"Order Growth: {revenue_metrics['order_growth_rate']:+.2f}%")
    print(f"AOV Growth: {revenue_metrics['aov_growth_rate']:+.2f}%")

In [ ]:
# Monthly revenue trend visualization
visualizer = MetricsVisualizer(business_report)
revenue_fig = visualizer.plot_revenue_trend(figsize=(14, 8))
plt.show()

# Monthly trends analysis
monthly_trends = business_report['monthly_trends']
print(f"\nMONTHLY PERFORMANCE:")
print(f"Best Revenue Month: Month {monthly_trends.loc[monthly_trends['revenue'].idxmax(), 'month']} (${monthly_trends['revenue'].max():,.0f})")
print(f"Lowest Revenue Month: Month {monthly_trends.loc[monthly_trends['revenue'].idxmin(), 'month']} (${monthly_trends['revenue'].min():,.0f})")
print(f"Average Monthly Growth: {monthly_trends['revenue_growth'].mean():.2f}%")
print(f"Revenue Volatility (Std Dev): ${monthly_trends['revenue'].std():,.0f}")

### 5.2 Product Category Performance {#product-analysis}

Understanding which product categories drive the most revenue and identifying growth opportunities.

In [ ]:
# Product category analysis
if 'error' not in business_report['product_performance']:
    product_data = business_report['product_performance']
    
    print(f"TOP PRODUCT CATEGORIES - {ANALYSIS_YEAR}")
    print("=" * 50)
    
    top_categories = product_data['top_categories'].head(10)
    for idx, row in top_categories.iterrows():
        print(f"{row['product_category_name']:<25} ${row['total_revenue']:>10,.0f} ({row['revenue_share']:>5.1f}%)")
    
    # Category performance visualization
    category_fig = visualizer.plot_category_performance(top_n=10, figsize=(14, 10))
    plt.show()
    
    # Category insights
    total_categories = len(product_data['all_categories'])
    top_5_share = top_categories.head(5)['revenue_share'].sum()
    
    print(f"\nCATEGORY SUMMARY:")
    print(f"Total Product Categories: {total_categories}")
    print(f"Top 5 Categories Revenue Share: {top_5_share:.1f}%")
else:
    print("Product category data not available for analysis")

### 5.3 Geographic Performance Analysis {#geographic-analysis}

Analyzing sales performance across different geographic regions to identify market opportunities.

In [ ]:
# Geographic analysis
geo_data = business_report['geographic_performance']

if 'error' not in geo_data.columns:
    print(f"GEOGRAPHIC PERFORMANCE - {ANALYSIS_YEAR}")
    print("=" * 50)
    
    # Top performing states
    top_states = geo_data.head(10)
    print("TOP 10 STATES BY REVENUE:")
    for idx, row in top_states.iterrows():
        print(f"{row['state']:<3} ${row['revenue']:>10,.0f} ({row['orders']:>5,} orders, AOV: ${row['avg_order_value']:>7,.0f})")
    
    # Geographic heatmap
    geo_fig = visualizer.plot_geographic_heatmap()
    geo_fig.show()
    
    # Geographic summary
    total_states = len(geo_data)
    top_5_revenue = top_states.head(5)['revenue'].sum()
    total_revenue = geo_data['revenue'].sum()
    top_5_share = (top_5_revenue / total_revenue) * 100
    
    print(f"\nGEOGRAPHIC SUMMARY:")
    print(f"States with Sales: {total_states}")
    print(f"Top 5 States Revenue Share: {top_5_share:.1f}%")
    print(f"Highest AOV State: {geo_data.loc[geo_data['avg_order_value'].idxmax(), 'state']} (${geo_data['avg_order_value'].max():,.0f})")
else:
    print("Geographic data not available for analysis")

### 5.4 Customer Experience Analysis {#customer-analysis}

Evaluating customer satisfaction through review scores and delivery performance metrics.

In [ ]:
# Customer satisfaction analysis
satisfaction_metrics = business_report['customer_satisfaction']

if 'error' not in satisfaction_metrics:
    print(f"CUSTOMER SATISFACTION ANALYSIS - {ANALYSIS_YEAR}")
    print("=" * 50)
    print(f"Average Review Score: {satisfaction_metrics['avg_review_score']:.2f}/5.0")
    print(f"Total Reviews: {satisfaction_metrics['total_reviews']:,}")
    print(f"5-Star Reviews: {satisfaction_metrics['score_5_percentage']:.1f}%")
    print(f"4+ Star Reviews: {satisfaction_metrics['score_4_plus_percentage']:.1f}%")
    print(f"1-2 Star Reviews: {satisfaction_metrics['score_1_2_percentage']:.1f}%")
    
    # Review distribution visualization
    review_fig = visualizer.plot_review_distribution(figsize=(12, 6))
    plt.show()
else:
    print("Customer satisfaction data not available for analysis")

In [ ]:
# Delivery performance analysis
delivery_metrics = business_report['delivery_performance']

if 'error' not in delivery_metrics:
    print(f"DELIVERY PERFORMANCE ANALYSIS - {ANALYSIS_YEAR}")
    print("=" * 50)
    print(f"Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
    print(f"Median Delivery Time: {delivery_metrics['median_delivery_days']:.1f} days")
    print(f"Fast Delivery (3 days or fewer): {delivery_metrics['fast_delivery_percentage']:.1f}%")
    print(f"Slow Delivery (more than 7 days): {delivery_metrics['slow_delivery_percentage']:.1f}%")
    
    if 'error' not in satisfaction_metrics:
        print(f"\nDELIVERY AND SATISFACTION:")
        print(f"Average satisfaction score: {satisfaction_metrics['avg_review_score']:.2f}")
        print(f"Fast delivery rate: {delivery_metrics['fast_delivery_percentage']:.1f}%")
else:
    print("Delivery performance data not available for analysis")

## 6. Summary of Key Observations {#summary}

A factual summary of the key findings from the analysis.

In [ ]:
# Generate executive summary
print(f"EXECUTIVE SUMMARY - {ANALYSIS_YEAR} BUSINESS PERFORMANCE")
print("=" * 60)

# Key metrics summary
revenue_metrics = business_report['revenue_metrics']
print(f"\nFINANCIAL PERFORMANCE:")
print(f"   Total Revenue: ${revenue_metrics['total_revenue']:,.0f}")
print(f"   Total Orders: {revenue_metrics['total_orders']:,}")
print(f"   Average Order Value: ${revenue_metrics['average_order_value']:,.0f}")

if 'revenue_growth_rate' in revenue_metrics:
    print(f"   Revenue Growth: {revenue_metrics['revenue_growth_rate']:+.1f}% vs {COMPARISON_YEAR}")

# Product insights
if 'error' not in business_report['product_performance']:
    top_category = business_report['product_performance']['top_categories'].iloc[0]
    print(f"\nPRODUCT PERFORMANCE:")
    print(f"   Top Category: {top_category['product_category_name']} (${top_category['total_revenue']:,.0f})")
    print(f"   Category Market Share: {top_category['revenue_share']:.1f}%")

# Geographic insights
geo_data = business_report['geographic_performance']
if 'error' not in geo_data.columns:
    top_state = geo_data.iloc[0]
    print(f"\nGEOGRAPHIC PERFORMANCE:")
    print(f"   Top Market: {top_state['state']} (${top_state['revenue']:,.0f})")
    print(f"   Active Markets: {len(geo_data)} states")

# Customer experience
if 'error' not in business_report['customer_satisfaction']:
    satisfaction = business_report['customer_satisfaction']
    print(f"\nCUSTOMER EXPERIENCE:")
    print(f"   Average Rating: {satisfaction['avg_review_score']:.1f}/5.0")
    print(f"   4+ Star Reviews: {satisfaction['score_4_plus_percentage']:.0f}%")

if 'error' not in business_report['delivery_performance']:
    delivery = business_report['delivery_performance']
    print(f"   Average Delivery: {delivery['avg_delivery_days']:.1f} days")
    print(f"   Fast Delivery (3 days or fewer): {delivery['fast_delivery_percentage']:.0f}%")

print(f"\n" + "=" * 60)

### Areas for Further Investigation

Based on the data presented above, the following areas may warrant deeper analysis:

In [ ]:
# Data-driven observations for further investigation
print("AREAS FOR FURTHER INVESTIGATION")
print("=" * 50)

observations = []

# Revenue observations
if 'revenue_growth_rate' in revenue_metrics:
    growth = revenue_metrics['revenue_growth_rate']
    observations.append(
        f"Revenue changed by {growth:+.1f}% year-over-year "
        f"(${revenue_metrics['total_revenue']:,.0f} vs "
        f"${revenue_metrics['previous_year_revenue']:,.0f})")

# Product concentration observations
if 'error' not in business_report['product_performance']:
    top_5_share = business_report['product_performance']['top_categories'].head(5)['revenue_share'].sum()
    observations.append(
        f"Top 5 product categories account for {top_5_share:.1f}% of total revenue")

# Geographic observations
geo_data = business_report['geographic_performance']
if 'error' not in geo_data.columns:
    observations.append(
        f"Sales span {len(geo_data)} states, with the top 5 generating "
        f"{(geo_data.head(5)['revenue'].sum() / geo_data['revenue'].sum() * 100):.1f}% of revenue")

# Customer satisfaction observations
if 'error' not in business_report['customer_satisfaction']:
    sat = business_report['customer_satisfaction']
    observations.append(
        f"Average review score is {sat['avg_review_score']:.2f}/5.0, "
        f"with {sat['score_4_plus_percentage']:.1f}% of reviews at 4+ stars "
        f"and {sat['score_1_2_percentage']:.1f}% at 1-2 stars")

# Delivery observations
if 'error' not in business_report['delivery_performance']:
    del_metrics = business_report['delivery_performance']
    observations.append(
        f"Average delivery takes {del_metrics['avg_delivery_days']:.1f} days, "
        f"with {del_metrics['fast_delivery_percentage']:.1f}% delivered in 3 days or fewer "
        f"and {del_metrics['slow_delivery_percentage']:.1f}% taking more than 7 days")

for i, obs in enumerate(observations, 1):
    print(f"{i}. {obs}")

print("\n" + "=" * 50)
print(f"Analysis completed for {ANALYSIS_YEAR}")
if COMPARISON_YEAR:
    print(f"Comparison baseline: {COMPARISON_YEAR}")
print(f"Generated on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

## Analysis Configuration Summary

This notebook provides a comprehensive, configurable framework for e-commerce business analysis. Key features:

- **Configurable Time Periods**: Easily adjust analysis and comparison years
- **Modular Architecture**: Reusable data loading and metrics calculation modules
- **Comprehensive Metrics**: Revenue, product, geographic, and customer experience analysis
- **Visual Insights**: Interactive charts and geographic visualizations

### Next Steps

1. **Regular Monitoring**: Schedule monthly/quarterly runs of this analysis
2. **Deeper Segmentation**: Analyze specific customer segments or product lines
3. **Predictive Analytics**: Implement forecasting models for future planning
4. **A/B Testing**: Design experiments to test strategic recommendations
5. **Real-time Dashboards**: Create live dashboards for ongoing monitoring

---

*This analysis framework is designed to be easily maintained and extended for future business intelligence needs.*